In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

# Load your data
df = pd.read_csv('mucolytics.csv')

# Convert 'Sold_date' to datetime format
df['Sold_date'] = pd.to_datetime(df['Sold_date'], format='%m/%d/%y')

# Set 'Sold_date' as the index
df.set_index('Sold_date', inplace=True)

# Aggregate sales data on a weekly basis for each product
weekly_sales = df.groupby([pd.Grouper(freq='W'), 
'Product_details']).agg(total_sold=('Sold_quantity', 'sum')).reset_index()

# Apply one-hot encoding to 'Product_details'
weekly_sales_encoded = pd.concat([weekly_sales, pd.get_dummies(weekly_sales['Product_details'], 
prefix='product')], axis=1).drop('Product_details', axis=1)

# Extract year and week number from 'Sold_date' for temporal features
weekly_sales_encoded['year'] = weekly_sales_encoded['Sold_date'].dt.year
weekly_sales_encoded['week_of_year'] = weekly_sales_encoded['Sold_date'].dt.isocalendar().week

# Normalize the temporal features 
scaler = MinMaxScaler()
features_columns = ['year', 'week_of_year'] + [col for col in weekly_sales_encoded.columns 
if col.startswith('product')]
weekly_sales_encoded[features_columns] = scaler.fit_transform(weekly_sales_encoded[features_columns])

# Split the data into features and target
X = weekly_sales_encoded[features_columns].values
y = weekly_sales_encoded['total_sold'].values

# Adjusting the dataset split into 70% training, 15% validation, and 15% testing
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42)  # 0.1765 of 85% is roughly 15% of the whole

# Define the model architecture
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(1, activation='relu')    # Output layer
])


# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error')

# Train the model with the validation data
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=10)

# Evaluate the model
predictions = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'RMSE on test set: {rmse}')

# Prepare forecasting features for Feb 1 to 8, 2024, correctly this time
forecast_results = []
for product_col in [col for col in features_columns if col.startswith('product')]:
    forecast_period_features = np.zeros((1, len(features_columns)))
    # Find the index of the year, week_of_year, and current product column
    year_index = features_columns.index('year')
    week_of_year_index = features_columns.index('week_of_year')
    product_index = features_columns.index(product_col)
    
    # Normalize year and week_of_year values
    forecast_period_features[0, year_index] = (2024 - scaler.data_min_[year_index]) / (scaler.data_max_[year_index] - scaler.data_min_[year_index])
    forecast_period_features[0, week_of_year_index] = (5 - scaler.data_min_[week_of_year_index]) / (scaler.data_max_[week_of_year_index] - scaler.data_min_[week_of_year_index])
    forecast_period_features[0, product_index] = 1  # Activate current product

    # Predict and store the result
    predicted_quantity = model.predict(forecast_period_features).flatten()[0]
    product_name = product_col.replace('product_', '')  # Assuming prefix 'product_' is used
    forecast_results.append((product_name, predicted_quantity))

# Display forecasted quantities for each product in a DataFrame
forecast_df = pd.DataFrame(forecast_results, columns=['Product', 'Forecasted Quantity']).sort_values(by='Forecasted Quantity', ascending=False)
forecast_df.to_excel('mucolytics_forecast_results.xlsx', index=False)


Epoch 1/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 567us/step - loss: 1057.1661 - val_loss: 336.8857
Epoch 2/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 351us/step - loss: 475.5294 - val_loss: 324.6775
Epoch 3/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 349us/step - loss: 330.1343 - val_loss: 350.0595
Epoch 4/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 347us/step - loss: 292.4099 - val_loss: 318.7081
Epoch 5/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 347us/step - loss: 299.9059 - val_loss: 336.8165
Epoch 6/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 350us/step - loss: 771.6906 - val_loss: 355.0681
Epoch 7/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 346us/step - loss: 391.9254 - val_loss: 338.8934
Epoch 8/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 348us/step - loss: 500.5500 - val_loss: 327.8043
Epoch 9/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 347us/step - loss: 276.4276 - val_loss: 319.7792
Epoch 10/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 346us/step - loss: 393.9925 - val_loss: 317.9946
Epoch 11/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 346us/step - loss: 497.

184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 356us/step - loss: 302.6140 - val_loss: 232.8800
Epoch 66/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 348us/step - loss: 205.0503 - val_loss: 247.7634
Epoch 67/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 349us/step - loss: 270.4501 - val_loss: 231.0181
Epoch 68/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 350us/step - loss: 200.6166 - val_loss: 229.2643
Epoch 69/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 351us/step - loss: 251.4581 - val_loss: 254.0579
Epoch 70/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 349us/step - loss: 258.7672 - val_loss: 235.9436
Epoch 71/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 346us/step - loss: 344.7305 - val_loss: 209.2370
Epoch 72/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 349us/step - loss: 294.4486 - val_loss: 255.2480
Epoch 73/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 346us/step - loss: 221.4704 - val_loss: 223.0710
Epoch 74/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 346us/step - loss: 277.1362 - val_loss: 219.9489
Epoch 75/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 348us/step - loss: 248.7963 